In [11]:
!pip install spacy pandas scikit-learn
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 840.2 kB/s eta 0:00:15
     -- ------------------------------------- 0.8/12.8 MB 1.0 MB/s eta 0:00:12
     ---- ----------------------------------- 1.3/12.8 MB 1.2 MB/s eta 0:00:10
     ----- ---------------------------------- 1.8/12.8 MB 1.4 MB/s eta 0:00:08
     ------ --------------------------------- 2.1/12.8 MB 1.4 MB/s eta 0:00:08
     ------- -------------------------------- 2.4/12.8 MB 1.4 MB/s eta 0:00:08
     -------- ------------------------------- 2.6/12.8 MB 1.4 MB/s eta 0:00:08
     --------- ------------------------------ 2.9/12.8 MB 1.3 MB/s eta 0:00:08
     --------- ------------------------------ 2.9/12.8 MB 1.3 MB/s eta 0:

In [12]:
pip install "numpy<2"


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [13]:
import pandas as pd
df = pd.read_csv("Reviews.csv")
df = df.dropna(subset=['Text'])
df = df.head(1000)
df = df[['Id', 'ProductId', 'UserId', 'ProfileName', 'Score', 'Time', 'Summary', 'Text']]


In [14]:
!pip install spacy


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.3.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.2-cp312-cp312-win_amd64.whl (12.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.3.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.2 which is incompatible.
scipy 1.13.1 requires numpy<2.3,>=1.22.4, but you have numpy 2.3.2 which is incompatible.


In [15]:
!pip install --upgrade numpy h5py spacy
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
def preprocess(text):
  doc = nlp(text.lower())
  tokens =[token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
  return " ".join(tokens)
df['Cleaned_Text']=df['Text'].apply(preprocess)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer =TfidfVectorizer()
X=vectorizer.fit_transform(df['Cleaned_Text'])

In [ ]:
def preprocess_query(query):
  return preprocess(query)
def vectorize_query(query):
  cleaned_query = preprocess_query(query)
  return vectorizer.transform([cleaned_query])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def retrieve_similar_reviews(query, top_k=5):
    query_vec = vectorize_query(query)
    similarities = cosine_similarity(query_vec, X).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    top_reviews = df.iloc[top_indices]
    top_scores = similarities[top_indices]
    results = []
    for i, row in enumerate(top_reviews.itertuples()):
        results.append({
            'Score': row.Score,
            'Summary': row.Summary,
            'Review': row.Text,
            'Similarity': round(top_scores[i], 3)
        })
    return results


In [ ]:
query = "great product with fast shipping"
matches = retrieve_similar_reviews(query)
for i, match in enumerate(matches, 1):
    print(f"\nMatch #{i}")
    print(f"Similarity: {match['Similarity']}")
    print(f"Score: {match['Score']}")
    print(f"Summary: {match['Summary']}")
    print(f"Review: {match['Review']}")
